# The parameter fit

In [1]:
import pandas as pd
import numpy as np

from scipy.optimize import minimize
from data.canada import nrcan_tech_shares_df

from batch import BatchResult

historic_tech_shares = nrcan_tech_shares_df.copy()
historic_tech_shares.index = historic_tech_shares.index.swaplevel()

province = "Ontario"

h_tech_shares = historic_tech_shares.loc[province, :] / 100
att_mode_table = h_tech_shares.copy()


In [2]:
from parameter_fit import fit_attitudes

results = fit_attitudes(
    0.6, 0.2, att_mode_table=att_mode_table, N=200, n_fit_iterations=20, ts_step_length="M"
    )

results_dir=WindowsPath('c:/src/canadopt/abetam/results/feature_utility_distribution/932b6ad8f43158e95c2ce6d6a203bc82') does not exist. Running model.
p_mode=0.60, peer_eff=0.20, i=00, current_abs_diff_sum=6.596, best_abs_diff_sum=1000000000000.000
p_mode=0.60, peer_eff=0.20, i=00, diff.abs().sum()=:
Electric furnace                1.687425
Gas furnace                     3.367216
Heat pump                       0.868044
Oil furnace                     0.572452
Wood or wood pellets furnace    0.101021
dtype: float64
results_dir=WindowsPath('c:/src/canadopt/abetam/results/feature_utility_distribution/9c6b8334227ee87a1a27ff80e0de04c6') does not exist. Running model.
p_mode=0.60, peer_eff=0.20, i=01, current_abs_diff_sum=6.430, best_abs_diff_sum=6.596
p_mode=0.60, peer_eff=0.20, i=01, diff.abs().sum()=:
Electric furnace                1.654425
Gas furnace                     3.283216
Heat pump                       0.819044
Oil furnace                     0.572452
Wood or wood pellets fur

In [4]:
shares_df, fitted_tech_shares, all_att_modes, best_modes = results

In [5]:
historic_tech_shares_l = (historic_tech_shares.loc["Ontario",:]/100).melt(ignore_index=False).reset_index()
historic_tech_shares_l["iteration"] = "historic"
tech_shares_df = pd.concat([historic_tech_shares_l, fitted_tech_shares])
tech_shares_df["var_type"] = "tech_share"
tech_shares_df

,year,variable,value,iteration,province,peer_eff,p_mode,var_type
0,2000.00,Electric furnace,0.160830,historic,NaN,NaN,NaN,tech_share
1,2001.00,Electric furnace,0.158447,historic,NaN,NaN,NaN,tech_share
2,2002.00,Electric furnace,0.156958,historic,NaN,NaN,NaN,tech_share
3,2003.00,Electric furnace,0.155691,historic,NaN,NaN,NaN,tech_share
4,2004.00,Electric furnace,0.154526,historic,NaN,NaN,NaN,tech_share
...,...,...,...,...,...,...,...,...
4855,2019.00,Heat pump,0.263000,11,Ontario,0.2,0.6,tech_share
4856,2019.25,Heat pump,0.263000,11,Ontario,0.2,0.6,tech_share
4857,2019.50,Heat pump,0.273000,11,Ontario,0.2,0.6,tech_share
4858,2019.75,Heat pump,0.275000,11,Ontario,0.2,0.6,tech_share


In [6]:
att_modes = all_att_modes.melt(ignore_index=False, id_vars=["province","iteration", "p_mode", "peer_eff",]).reset_index()
att_modes["var_type"] = "att_mode"

data = pd.concat([tech_shares_df, att_modes])

In [8]:
import plotly.express as px
# fig = px.line(fitted_tech_shares, x="year", y="value", color="variable", line_dash="iteration", template="plotly")
fig = px.line(data, x="year", y="value", color="variable", line_dash="iteration", template="plotly", facet_row="var_type")
fig.update_traces(opacity=.2, )
def historic_trace(trace):
    return "historic" in trace.name
fig.update_traces(line=dict(width=3, dash="solid"), opacity=1, selector = historic_trace)
fig.update_yaxes(matches=None)

In [8]:


def diff_btwn_model_historic(att_modes_array, N=100, province="Ontario", p_mode=0.5, peer_eff=0.2):
    att_modes = flat_array_2_table(att_modes_array)
    batch_parameters = {
        "N": [N],
        "province": [province],
        "random_seed": range(20, 25),
        "start_year": 2000,
        "tech_att_mode_table": [att_modes],
        "n_segregation_steps": [40],
        "interact": [False],
        "price_weight_mode": [p_mode],
        "ts_step_length":["W"],
        "peer_effect_weight": [peer_eff]
    }
    b_result = BatchResult.from_parameters(batch_parameters, display_progress=False)
    model_shares = (
        b_result.tech_shares_df.groupby(["province", "year"])
        .mean()
        .drop("RunId", axis=1)
    )
    full_years = range(2000, 2021)
    diff = (h_tech_shares - model_shares.loc[(province, full_years), :]).loc[
        province, :
    ]
    abs_diff = diff.abs().sum().sum()
    print(abs_diff)
    return abs_diff

def flatten_table(table):
    return table.values.flatten()

def flat_array_2_table(array):
    cols = ['Electric furnace', 'Gas furnace', 'Heat pump', 'Oil furnace',
       'Wood or wood pellets furnace']
    return pd.DataFrame(array.reshape((21,5)), columns=cols, index=range(2000,2021))


x0=flatten_table(h_tech_shares.copy())
bounds = [*[(0.05, 0.95)]*len(x0)]
# other methods to try:
# L-BFGS-T is the default and doesn't appear to converge after > 10 iterations
# [ ] Powell
# [ ] TNC
# [ ] trust-constr
# [ ] COBYLA

# res = minimize(diff_btwn_model_historic, x0=x0, bounds=bounds, options={"maxiter":10, "disp":True}, method="SLSQP")

,Electric furnace,Gas furnace,Heat pump,Oil furnace,Wood or wood pellets furnace
2000,0.160830,0.697388,0.044867,0.086909,0.010005
2001,0.158447,0.702676,0.046687,0.082539,0.009591
2002,0.156958,0.710500,0.048591,0.074012,0.009830
2003,0.155691,0.714465,0.050494,0.068538,0.010527
2004,0.154526,0.716888,0.052200,0.065677,0.010421
2005,0.154173,0.716791,0.053854,0.063713,0.011024
2006,0.154680,0.718629,0.055267,0.060159,0.010567
2007,0.155646,0.718490,0.056847,0.056612,0.011040
2008,0.157546,0.716876,0.058311,0.054277,0.010718
2009,0.159955,0.713991,0.059811,0.052498,0.010178


# Analysing parameter fit results

In [ ]:
import pandas as pd
import plotly.express as px
from pathlib import Path
# results_dir = "results/fitting/2024.02.26-14.39"
# results_dir = "results/fitting/2024.03.12-16.03"
# results_dir = "results/fitting/2024.03.13-16.19"
# results_dir = "results/fitting/2024.04.05-14.52"
# results_dir = "results/fitting/2024.07.15-06.50"
# results_dir = "results/fitting/2024.08.08-13.19"
results_dir = "results/fitting/2024.08.09-13.38"

def file_in_dir_like(dir, name_frag, suffix=".csv"):
    matches = list(Path(results_dir).glob(f"*{name_frag}*{suffix}"))
    if len(matches) != 1:
        raise ValueError(f"found {matches}, proceeding not defined")
    return matches[0]
att_mode_table = pd.read_csv(file_in_dir_like(results_dir,"all_attitude_modes"))

# att_mode_table[["p_mode","gut"]] = att_mode_table[["p_mode","gut"]].round(2)
att_mode_table["p_mode"] = att_mode_table["p_mode"].round(2)

att_mode_table.head()

In [ ]:
best_modes = pd.read_csv(file_in_dir_like(results_dir, "best_modes"))
best_modes[["p_mode"]] = best_modes[["p_mode"]].round(2)

best_modes.head()

In [ ]:
# best_modes[["best_abs_diff", "province", "p_mode",]].groupby(["province","p_mode"]).mean().reset_index().pivot(index=["province"], columns="p_mode", values="best_abs_diff")
best_modes[["best_abs_diff", "province", "p_mode","peer_eff"]].groupby(["province","p_mode","peer_eff"]).mean().reset_index().pivot(index=["peer_eff"], columns="p_mode", values="best_abs_diff")

In [ ]:
# top_12 = best_modes.groupby(["p_mode"])[["best_abs_diff"]].min().sort_values(by="best_abs_diff").reset_index().loc[:4,:]
top_12 = best_modes.groupby(["p_mode","peer_eff"])[["best_abs_diff"]].min().sort_values(by="best_abs_diff").reset_index().loc[:4,:]
top_12

In [ ]:
# gut = top_12['gut'][0]
p_mode = top_12['p_mode'][0]
# best_modes.query(f"gut=={gut} and p_mode=={p_mode}").iloc[:,:-4].set_index("year").loc[2020,:]
best_modes.query(f"p_mode=={p_mode}").iloc[:,:].set_index("year").loc[2020,:]

In [ ]:
top_p_modes = top_12['p_mode'].to_list()
# top_guts = top_12['gut'].to_list()
asc_p_mode = sorted(top_12["p_mode"].unique())
# asc_gut = sorted(top_12["gut"].unique())
asc_p_mode

In [ ]:
# att_mode_table["gut and p_mode"] = att_mode_table[["gut","p_mode"]].values.tolist()
# att_mode_table["gut and p_mode"] = att_mode_table["gut and p_mode"].apply(tuple)
att_mode_table.head()

In [ ]:

atts = att_mode_table.query(f"variable!='province'").set_index(["p_mode","year"]).loc[asc_p_mode,:,:].reset_index()
atts["value"] = atts["value"].astype(float)
fig = px.line(atts, x="year", y="value", color="variable", facet_col="p_mode", facet_row="peer_eff", line_dash="iteration", template="plotly", height=700, category_orders={"p_mode":asc_p_mode, })#"gut":asc_gut})
# fig = px.line(atts, x="year", y="value", color="variable", facet_col="p_mode", line_dash="iteration", template="plotly", height=700,)# category_orders={"p_mode":asc_p_mode, "gut":asc_gut})
fig.update_traces(opacity=.2, )
fig.update_layout(yaxis_title="Mode")
fig.write_html(f"{results_dir}/fitted_att_modes.html")
fig

In [ ]:
# best_modes["gut and p_mode"] = best_modes[["gut","p_mode"]].values.tolist()
# best_modes["gut and p_mode"] = best_modes["gut and p_mode"].apply(tuple)

# best_modes_long = best_modes.melt(id_vars=["year","province","p_mode","peer_eff","best_abs_diff","gut and p_mode"])#.set_index(["gut and p_mode","year"]).loc[top_gut_p_modes,:,:].reset_index()
best_modes_long = best_modes.melt(id_vars=["year","province","p_mode","peer_eff","best_abs_diff"]).reset_index()#.set_index(["gut and p_mode","year"]).loc[top_gut_p_modes,:,:].reset_index()
# best_modes_long = best_modes.melt(id_vars=["year","province","p_mode","best_abs_diff"]).set_index(["p_mode","year"]).loc[asc_p_mode,:,:].reset_index()
fig = px.line(best_modes_long, x="year", y="value", color="variable", facet_col="p_mode", facet_row="peer_eff", template="plotly", height=700)
# fig = px.line(best_modes_long, x="year", y="value", color="variable", facet_col="p_mode", template="plotly", height=700)
fig.update_layout(yaxis_title="Mode")
fig.write_html(f"{results_dir}/fitted_att_modes_best_only.html")
fig


In [ ]:
tech_shares = pd.read_csv(file_in_dir_like(results_dir, "historic_tech"), index_col=0)
# tech_shares[["gut","p_mode"]] = tech_shares[["gut","p_mode"]].round(2)
# tech_shares["gut and p_mode"] = tech_shares[["gut","p_mode"]].values.tolist()
# tech_shares["gut and p_mode"] = tech_shares["gut and p_mode"].apply(tuple)
# tech_shares = tech_shares.set_index(["gut and p_mode","year"]).loc[top_gut_p_modes,:,:].reset_index()
tech_shares.head()

In [ ]:
import numpy as np
contains_val = ~top_12.pivot(columns="p_mode",index="peer_eff").isna().values
# contains_val = ~top_12.pivot(columns="p_mode",index="gut").isna().values
indcs = list(zip(*np.where(contains_val==True)))
indcs

In [ ]:
from data.canada import nrcan_tech_shares_df
from config import TECHNOLOGY_COLORS
# fig = px.line(tech_shares, x="year", y="value", color="variable", facet_col="p_mode",  line_dash="iteration", template="plotly", height=700, color_discrete_map=TECHNOLOGY_COLORS)
fig = px.line(tech_shares, x="year", y="value", color="variable", facet_col="p_mode", facet_row="peer_eff", line_dash="iteration", template="plotly",  height=700, color_discrete_map=TECHNOLOGY_COLORS)#category_orders={"p_mode":asc_p_mode, "gut":asc_gut},
fig.for_each_trace(lambda t: t.update(showlegend=False))

fig.update_traces(opacity=0.2)

province="Ontario"
historic_tech_shares = nrcan_tech_shares_df.copy()
historic_tech_shares.index = historic_tech_shares.index.swaplevel()
h_tech_shares = historic_tech_shares.loc[province, :] / 100

h_tech_shares_long = h_tech_shares.melt(ignore_index=False)

hist_fig = px.line(h_tech_shares_long.reset_index(), x="year", y="value", color="variable", template="plotly", color_discrete_map=TECHNOLOGY_COLORS)


legend_names = []
# for idx in indcs:
for i,idx in enumerate(indcs):
    for trace in hist_fig.data:
        if trace.name in legend_names:
            showlegend = False
        else: 
            showlegend = True
            legend_names.append(trace.name)
        trace.showlegend = showlegend
        # print(np.array(indcs).max(axis=0)[0]+1-idx[0])
        # fig.add_trace(trace, row=1, col=i+1)
        print(indcs)
        fig.add_trace(trace, row=np.array(indcs).max(axis=0)[0]+1-idx[0], col=idx[1]+1)
fig.update_layout(yaxis_title="Adoption Levels")
fig.write_html(f"{results_dir}/fitted_adoption_levels.html")
fig

# Overlay batch run with historic adoption

In [ ]:
import matplotlib.pyplot as plt
from config import TECHNOLOGY_COLORS
import seaborn as sns
# ax = result.tech_shares_fig()
# added_historic_traces = False
top_12["p_mode"] = top_12["p_mode"].round(2)
tech_shares["p_mode"] = tech_shares["p_mode"].round(2)
best_mode = top_12.loc[0,"p_mode"]

ax = sns.lineplot(tech_shares.query(f"p_mode=={best_mode}"), x="year", y="value", hue="variable", palette=TECHNOLOGY_COLORS)

In [ ]:
from data.canada import nrcan_tech_shares_df
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import config
added_historic_traces=False
historic_tech_shares = nrcan_tech_shares_df.copy()
historic_tech_shares.index = historic_tech_shares.index.swaplevel()
if not added_historic_traces:
    for tech in historic_tech_shares.columns:
        y = historic_tech_shares.loc[province, tech]/100
        added_historic_traces = True
        ax.plot(y.index,y.values, "--", color=config.TECHNOLOGY_COLORS[tech],)

# new_title = 'Heating technologies'
ax.legend()


custom_lines = [Line2D([0], [0], linestyle="--", color="black", lw=1),
                Line2D([0], [0], color="black", lw=1),
                ]
ax.legend(custom_lines, ['historic', 'modelled'],)
ax.set_xticks(range(2000,2021,5), labels=range(2000,2021,5))
ax.figure.savefig(f"{results_dir}/fitted_vs_historic_adoption.svg")
ax.figure


In [ ]:
leg = ax.ax.get_legend()
ax.add_legend(leg, title="Heating technologies", loc=(0.5, 0.5))
ax.ax.figure


In [ ]:
# import seaborn as sns
# future_shares = pd.read_csv(file_in_dir_like(results_dir, "future"), index_col=0)
# # future_shares[["gut","p_mode"]] = future_shares[["gut","p_mode"]].round(2)
# future_shares[["p_mode"]] = future_shares[["p_mode"]].round(2)
# # future_shares["gut and p_mode"] = future_shares[["gut","p_mode"]].values.tolist()
# # future_shares["gut and p_mode"] = future_shares["gut and p_mode"].apply(tuple)
# # future_shares = future_shares.set_index(["gut and p_mode","year"]).loc[top_gut_p_modes,:,:].melt(ignore_index=False, id_vars=["RunId","province","p_mode","gut"]).reset_index()

# future_shares_l = future_shares.drop("province",axis=1).melt(id_vars=["RunId","p_mode","year"])
# ax = sns.relplot(future_shares_l, x="year", y="value", hue="variable", col="p_mode",row="gut")
# ax = sns.relplot(future_shares_l, x="year", y="value", hue="variable", col="p_mode", col_wrap=4)#,row="gut")
# ax.figure.savefig(f"{results_dir}/fitted_future_adoptions.png")